In [1]:
import os
os.chdir('..')

import numpy as np
import pandas as pd

from utils.mongoDB import MongoDBController
from dotenv import load_dotenv

load_dotenv('./.env.development')

True

In [2]:
mongo = MongoDBController(
    host=os.environ['mongodb_url'],
    port=int(os.environ['mongodb_port']), 
    username=os.environ['mongodb_user'], 
    password=os.environ['mongodb_password']
)

In [3]:
df = mongo.find_all('test_data', 'sales_performance_report')
df.head()

,_id,order_id,order_status,customer,order_date,order_quantity,sales,discount,discount_value,product_category,product_sub_category
0,658bf83028aa6a4cf06eb533,3,Order Finished,Muhammed Mac Intyre,2010-10-13,6,523080,0.04,20923,Office Supplies,Storage & Organization
1,658bf83028aa6a4cf06eb534,293,Order Finished,Barry French,2012-10-01,49,20246040,0.07,1417223,Office Supplies,Appliances
2,658bf83028aa6a4cf06eb535,483,Order Finished,Clay Rozendal,2011-07-10,30,9931519,0.08,794522,Technology,Telephones & Communication
3,658bf83028aa6a4cf06eb536,515,Order Finished,Carlos Soltero,2010-08-28,19,788540,0.08,63083,Office Supplies,Appliances
4,658bf83028aa6a4cf06eb537,613,Order Finished,Carl Jackson,2011-06-17,12,187080,0.03,5612,Office Supplies,Binders & Binder Accessories


In [5]:
df['_year'] = df['order_date'].dt.year
df['_month'] = df['order_date'].dt.month
df['_day'] = df['order_date'].dt.day
df.head()

,_id,order_id,order_status,customer,order_date,order_quantity,sales,discount,discount_value,product_category,product_sub_category,_year,_month,_day
0,658bf83028aa6a4cf06eb533,3,Order Finished,Muhammed Mac Intyre,2010-10-13,6,523080,0.04,20923,Office Supplies,Storage & Organization,2010,10,13
1,658bf83028aa6a4cf06eb534,293,Order Finished,Barry French,2012-10-01,49,20246040,0.07,1417223,Office Supplies,Appliances,2012,10,1
2,658bf83028aa6a4cf06eb535,483,Order Finished,Clay Rozendal,2011-07-10,30,9931519,0.08,794522,Technology,Telephones & Communication,2011,7,10
3,658bf83028aa6a4cf06eb536,515,Order Finished,Carlos Soltero,2010-08-28,19,788540,0.08,63083,Office Supplies,Appliances,2010,8,28
4,658bf83028aa6a4cf06eb537,613,Order Finished,Carl Jackson,2011-06-17,12,187080,0.03,5612,Office Supplies,Binders & Binder Accessories,2011,6,17


In [6]:
df_table_desc = mongo.find_all(os.environ['mongodb_table_descriptor'], 'test_data')
table_desc = df_table_desc.loc[df_table_desc['collection'] == 'sales_performance_report', 'description'].iloc[0]
print(table_desc)

You are provided with historical sales data from 2009 to 2012. This data contain 3 product category which are office supplies, technology, and furniture. Each category has several sub-categories. The company also runs promotional in the form of a discount. Attribute Information:      order_id : unique order number     order_status : status of the order, whether is finished or returned     customer : customer name     order_date : date of the order     order_quantity : the quantity on a particular order     sales : sales generated on a particular order, the value is in IDR(Indonesia Rupiah) currency     discount : a discount percentage     discount_value : a sales multiply by discount, the value is in IDR(Indonesia Rupiah) currency     product_category : a category of the product     product_sub_category : a subcategory from product category
This is the quantitative information of the table:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5499 entries, 0 to 5498
Data columns (total 10

# Range

In [7]:
# How much is the highest order quantity for a single sales?
df['order_quantity'].max()

50

In [8]:
# How much is the highest amount of discount value applied for a single sales?
df['discount_value'].max()

7441778

In [9]:
# What is the order id for the sales with the highest order quantity?
df.loc[df['order_quantity'].idxmax(), 'order_id']

7430

In [10]:
# How much is the lowest amount for sales?
df['sales'].min()

6460

In [11]:
# What is the name of the customer with the lowest order quantity?
df.loc[df['order_quantity'].argmin(), 'customer']

'Delfina Latchford'

# Arithmetic

In [12]:
# How much is the total order quantity for office supplies? (table related)
df.loc[df['product_category'] == 'Office Supplies', 'order_quantity'].sum()

78422

In [13]:
# How much is the mean discount for computer peripheral sales? (table related)
df.loc[df['product_sub_category'] == 'Computer Peripherals', 'discount'].mean()

0.04983957219251337

In [14]:
# How much is the sales average for the Technology product category?
df.loc[df['product_category'] == 'Technology', 'sales'].mean()

5799030.34609375

In [15]:
# How much is the difference between the highest and lowest order quantity?
df['order_quantity'].max() - df['order_quantity'].min()

49

In [16]:
# For the sales with the highest sales value, divide the sales by its order quantity.
idx = df['sales'].idxmax()
df.loc[idx, 'sales'] / df.loc[idx, 'order_quantity']

13701700.0

# Date

In [19]:
# Give me the range of product categories that are ordered between 2012-10-01 and 2012-10-31
df[(df['_year'] == 2012) & (df['_month'] == 10)]['product_category'].unique()

array(['Office Supplies', 'Furniture', 'Technology'], dtype=object)

In [22]:
# What was the most top selling product sub category in June 2011?
df[(df['_year'] == 2011) & (df['_month'] == 6)]['product_sub_category'].value_counts().idxmax()

'Paper'

In [24]:
# What is the total order quantity between June 2011 and June 2012?
df[((df['_year'] == 2011) & (df['_month'] >= 6)) | ((df['_year'] == 2012) & (df['_month'] <= 6))]['order_quantity'].sum()

37915

In [29]:
# Which year has the most total sales?
df.groupby('_year')['sales'].sum().idxmax()

2009

In [34]:
# How much was the least amount of total sales in a year?
df.groupby('_year')['sales'].sum().min()

4535204888

# Percentage

In [38]:
# Give me the distribution of each product category in percentage.
df['product_category'].value_counts() / df['product_category'].value_counts().sum()

product_category
Office Supplies    0.557556
Technology         0.232770
Furniture          0.209674
Name: count, dtype: float64

In [39]:
# What percentage of the total were order quantity in 2012 over all the years in the data?
df.loc[df['_year'] == 2012, 'order_quantity'].sum() / df['order_quantity'].sum()

0.25844538024696634

In [40]:
# What is the percentage of sales that were closed in the month of October over all the years?
df.loc[df['_month'] == 10].shape[0] / df.shape[0]

0.08492453173304237

In [49]:
# How many percentage of orders were returned? (table related)
df['order_status'].value_counts().loc['Order Returned'] / df['order_status'].value_counts().sum()

0.10401891252955082

In [50]:
# Give me the distribution of each product sub category in percentage.
df['product_sub_category'].value_counts() / df['product_sub_category'].value_counts().sum()

product_sub_category
Binders & Binder Accessories    0.139662
Paper                           0.133843
Computer Peripherals            0.102019
Office Furnishings              0.096381
Telephones & Communication      0.078014
Appliances                      0.068194
Pens & Art Supplies             0.065285
Chairs & Chairmats              0.054737
Storage & Organization          0.050191
Office Machines                 0.042553
Labels                          0.037461
Tables                          0.032733
Envelopes                       0.031642
Bookcases                       0.025823
Rubber Bands                    0.017640
Scissors, Rulers & Trimmers     0.013639
Copiers & Fax                   0.010184
Name: count, dtype: float64